In [1]:
def getData(dbname,table,code,startdate,enddate):
    import sqlite3
    dbName = dbname
    tableName = table
    conn = sqlite3.connect(dbName)
    c = conn.cursor()
    #code, date, close, high, low, open, volume
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append([i][0])
    conn.close()
    return dataList
print("getData loaded")

def slidingWindow(dataList, wide=7):
    newList = []
    for i in range(len(dataList) - wide):
        newList.append(dataList[i:i+wide])
    return newList
print("slidingWindow loaded")

############################# CHECK BUY SIGNALS ##############################
def checkPastDescend(partList, days=7):
    # Open > Close
    flag = False
    if partList[0][6] > partList[days-5][3] : flag = True
    return flag

def partList2OnC(partList):
    #Make partList into readable Open and Close
    O1 = partList[-4][6]
    O2 = partList[-3][6]
    O3 = partList[-2][6]
    C1 = partList[-4][3]
    C2 = partList[-3][3]
    C3 = partList[-2][3]
    return O1,O2,O3,C1,C2,C3

def checkTWS(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (C1 > O1) : flag = False
    if not (C2 > O2) : flag = False
    if not (C3 > O3) : flag = False
    if not ((O3 > O2) and (O2 > O1)) : flag = False
    if not ((O3 > O2) and (O2 > O1)) : flag = False
    if not (C1 > O2) : flag = False
    if not (C2 > O3) : flag = False
    return flag

def checkTIU(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (O1 > C1) : flag = False
    if not (O1 > C1) : flag = False
    if not (C3 > O1) : flag = False
    if not (O1 > C2) : flag = False
    if not (C2 > O2) : flag = False
    if not (O2 > C1) : flag = False
    if not (C3 > O3) : flag = False
    if not (O3 > C1) : flag = False
    return flag

def checkTOU(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (O1 > C1) : flag = False
    if not (C2 > O2) : flag = False
    if not (C3 > O3) : flag = False
    if not (C3 > C2) : flag = False
    if not (C2 > O1) : flag = False
    if not (O2 < C1) : flag = False
    if not (O3 > O2) : flag = False
    return flag

def checkMS(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (O1 > C1) : flag = False
    if not (C3 > O1) : flag = False
    if not (O1 > O3) : flag = False
    if not (C1 > O2) : flag = False
    if not (C1 > C2) : flag = False
    if not (O3 > O2) : flag = False
    if not (O3 > C2) : flag = False
    return flag

def checkBuySignal(partList,days):
    flag = True
    if not checkPastDescend(partList, days) : flag = False
    if checkTWS(partList) : flag = True
    if checkTIU(partList) : flag = True
    if checkTOU(partList) : flag = True
    if checkMS(partList) : flag = True
    if True == flag:
        return partList[-1]
    else:
        return False
    
print("checkBuySignal loaded")
############################# CHECK BUY SIGNALS ##############################
############################# CHECK SELL SIGNALS ##############################
def checkPastAscend(partList, days=7):
    # Open < Close
    flag = False
    if partList[0][6] < partList[days-5][3] : flag = True
    return flag

def checkTBC(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (O1 > C1) : flag = False
    if not (O2 > C2) : flag = False
    if not (O3 > C3) : flag = False
    if not (O1 > O2) : flag = False
    if not (O2 > O3) : flag = False
    if not (C1 > C2) : flag = False
    if not (C2 > C3) : flag = False
    if not (O2 > C1) : flag = False
    if not (O3 > C2) : flag = False
    return flag

def checkTID(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (O1 < C1) : flag = False
    if not (C2 < O2) : flag = False
    if not (C3 < O3) : flag = False
    if not (C1 > O2) : flag = False
    if not (O2 > O3) : flag = False
    if not (O1 < C2) : flag = False
    if not (C3 < C2) : flag = False
    return flag

def checkTOD(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (C2 < O2) : flag = False
    if not (C3 < O3) : flag = False
    if not (O2 > C1) : flag = False
    if not (C1 > O1) : flag = False
    if not (O1 > C2) : flag = False
    if not (O3 > C2) : flag = False
    if not (C2 > C3) : flag = False
    return flag

def checkES(partList):
    O1,O2,O3,C1,C2,C3 = partList2OnC(partList)
    flag = True
    if not (C1 > O1) : flag = False
    if not (C3 < O3) : flag = False
    if not (O3 < O1) : flag = False
    if not (C1 < C2) : flag = False
    if not (C1 < O2) : flag = False
    if not (O3 < C2) : flag = False
    if not (O3 < O2) : flag = False
    return flag

def checkSellSignal(partList,days):
    flag = True
    if not checkPastAscend(partList, days) : flag = False
    if checkTBC(partList) : flag = True
    if checkTID(partList) : flag = True
    if checkTOD(partList) : flag = True
    if checkES(partList) : flag = True
    if True == flag:
        return partList[-1]
    else:
        return False

print("checkSellSignal loaded")
############################# CHECK SELL SIGNALS ##############################

def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList
print("getListedCode loaded")

def createDatabase(dbName,tableName):
    import os
    if os.path.isfile(dbName):
        return "Database name already exists: ", dbName
    else:    
        import sqlite3
        sqlite_file = dbName
        conn = sqlite3.connect(sqlite_file)
        c = conn.cursor()
        c.execute('CREATE TABLE {tn} (indexColumn INTEGER PRIMARY KEY, code TEXT NOT NULL, date TEXT NOT NULL, close REAL, high REAL, low REAL, open REAL, volume REAL, action TEXT NOT NULL)'.format(tn=tableName)) 
        conn.commit()
        conn.close()
        return "Database",dbName," Successfully Created!"
print("createDatabase loaded")

def insertData(dbName, tableName, dataList):
    import sqlite3
    conn = sqlite3.connect(dbName)
    c = conn.cursor()
    count = 0
    for code, date, close, high, low, ope, vol, act in dataList:
        conn.execute("INSERT INTO twse (code, date, close, high, low, open, volume, action) VALUES (?,?,?,?,?,?,?,?)",(code, date, close, high, low, ope, vol, act))
        count = count + 1
    conn.commit()
    conn.close()
    return count
print("insertData loaded")

getData loaded
slidingWindow loaded
checkBuySignal loaded
checkSellSignal loaded
getListedCode loaded
createDatabase loaded
insertData loaded


In [ ]:
codeList = getListedCode("twse",4) #取得上市股票代碼 twse, 長度4碼
print("getListedCode", codeList[0:5])

dbName = "buysell.db"
tableName = "twse"
createDatabase(dbName,tableName)
print("Database",dbName,"Table",tableName,"created")

for code in codeList:
    actionData = []
    print("Start",code)
    dataList = getData("PriceVolData.db","twse",code,"2007-05-25","2017-05-25")
    slideDays = 10
    slideList = slidingWindow(dataList,slideDays)
    for partList in slideList:
        buysig = checkBuySignal(partList,slideDays)
        if buysig:
            actionData.append([buysig[1], buysig[2], buysig[3], buysig[4], buysig[5], buysig[6], buysig[7],"buy"])
        sellsig = checkSellSignal(partList,slideDays)
        if sellsig:
            actionData.append([sellsig[1], sellsig[2], sellsig[3], sellsig[4], sellsig[5], sellsig[6], sellsig[7],"sell"])
    insertData(dbName, tableName, actionData)
print("End")

取得股票代號總數: 913
getListedCode ['1101', '1102', '1103', '1104', '1108']
Database buysell.db Table twse created
Start 1101
Start 1102
Start 1103
Start 1104
Start 1108
Start 1109
Start 1110
Start 1201
Start 1203
Start 1210
Start 1213
Start 1215
Start 1216
Start 1217
Start 1218
Start 1219
Start 1220
Start 1225
Start 1227
Start 1229
Start 1231
Start 1232
Start 1233
Start 1234
Start 1235
Start 1236
Start 1256
Start 1262
Start 1301
Start 1303
Start 1304
Start 1305
Start 1307
Start 1308
Start 1309
Start 1310
Start 1312
Start 1313
Start 1314
Start 1315
Start 1316
Start 1319
Start 1321
Start 1323
Start 1324
Start 1325
Start 1326
Start 1337
Start 1338
Start 1339
Start 1340
Start 1402
Start 1409
Start 1410
Start 1413
Start 1414
Start 1416
Start 1417
Start 1418
Start 1419
Start 1423
Start 1432
Start 1434
Start 1435
Start 1436
Start 1437
Start 1438
Start 1439
Start 1440
Start 1441
Start 1442
Start 1443
Start 1444
Start 1445
Start 1446
Start 1447
Start 1449
Start 1451
Start 1452
Start 1453
Start 1454
S